<a href="https://colab.research.google.com/github/dongxiat/MyColabHub/blob/main/DepthFlow_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 1. Cài đặt các thư viện hệ thống cần thiết cho EGL
!apt-get update
!apt-get install -y libegl1-mesa-dev libgles2-mesa-dev

# 2. Cài đặt moderngl
!pip install moderngl

# 3. Tạo context và chỉ định rõ backend 'egl'
import moderngl

# Thay vì để tự động chọn, chúng ta ép nó dùng 'egl'
# Đây là dòng quan trọng nhất để sửa lỗi
ctx = moderngl.create_standalone_context(backend='egl')

# 4. In thông tin để kiểm tra
print(ctx.info)

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [15]:
import os
import re
import json
from glob import glob

# --- PHẦN CẤU HÌNH EGL CHO NVIDIA ---
# Mục tiêu: Tìm tệp cấu hình EGL của NVIDIA và đặt biến môi trường
# để buộc moderngl sử dụng nó.

# 1. Tìm tệp nvidia_egl.json
vendor_json_path = ''
for path in glob('/usr/share/glvnd/egl_vendor.d/*.json'):
    with open(path) as f:
        data = json.load(f)
        if data.get('library_path') and 'nvidia' in data['library_path']:
            vendor_json_path = path
            break

# 2. Nếu tìm thấy, đặt biến môi trường
if vendor_json_path:
    print(f"Đã tìm thấy file cấu hình EGL của NVIDIA tại: {vendor_json_path}")
    os.environ['__EGL_VENDOR_LIBRARY_FILENAMES'] = vendor_json_path
else:
    print("CẢNH BÁO: Không tìm thấy file cấu hình EGL của NVIDIA. Có thể sẽ quay lại dùng llvmpipe.")
    # Cài đặt thủ công nếu không tìm thấy tự động (dành cho các phiên bản Colab cũ hơn)
    !apt-get update -qq && apt-get install -y libegl1-mesa-dev > /dev/null


# --- PHẦN KHỞI TẠO MODERNGL ---

# 3. Cài đặt moderngl
!pip install moderngl > /dev/null
import moderngl

# 4. Tạo context, EGL giờ sẽ đọc biến môi trường và chọn đúng driver
print("\nĐang tạo context ModernGL...")
try:
    ctx = moderngl.create_standalone_context(backend='egl')
    print("Tạo context thành công!")

    # 5. In thông tin để kiểm tra
    print("\n--- Thông tin Context ---")
    print(f"GL_VENDOR: {ctx.info.get('GL_VENDOR')}")
    print(f"GL_RENDERER: {ctx.info.get('GL_RENDERER')}")
    print(f"GL_VERSION: {ctx.info.get('GL_VERSION')}")
    print("-------------------------")

    if 'nvidia' in ctx.info.get('GL_RENDERER', '').lower():
        print("\n✅ Thành công! ModernGL đang render bằng GPU của NVIDIA.")
    else:
        print("\n❌ Thất bại! ModernGL vẫn đang dùng llvmpipe. Hãy kiểm tra lại Bước 1.")

except Exception as e:
    print(f"\nĐã xảy ra lỗi khi tạo context: {e}")
    print("Hãy chắc chắn bạn đã thực hiện Bước 1 và bật GPU Runtime.")

CẢNH BÁO: Không tìm thấy file cấu hình EGL của NVIDIA. Có thể sẽ quay lại dùng llvmpipe.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)

Đang tạo context ModernGL...
Tạo context thành công!

--- Thông tin Context ---
GL_VENDOR: Mesa
GL_RENDERER: llvmpipe (LLVM 15.0.7, 256 bits)
GL_VERSION: 4.5 (Core Profile) Mesa 23.2.1-1ubuntu3.1~22.04.3
-------------------------

❌ Thất bại! ModernGL vẫn đang dùng llvmpipe. Hãy kiểm tra lại Bước 1.


In [ ]:
import os
import subprocess
import moderngl

# --- Bước 1: Xác minh GPU một cách chắc chắn ---
gpu_check = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
if "NVIDIA-SMI has failed" in gpu_check.stdout or gpu_check.returncode != 0:
    print("❌ LỖI NGHIÊM TRỌNG: Không phát hiện GPU NVIDIA.")
    print("Vui lòng vào 'Runtime' -> 'Change runtime type' để chọn một 'T4 GPU' và chạy lại cell này.")
else:
    print("✅ Bước 1: Đã xác nhận có GPU NVIDIA.")

    # --- Bước 2: Tìm file cấu hình EGL của NVIDIA trên toàn hệ thống ---
    print("\n⏳ Bước 2: Đang tìm kiếm file cấu hình EGL của NVIDIA...")
    # Lệnh tìm kiếm mạnh mẽ hơn
    find_cmd = 'find / -name "nvidia_egl.json" 2>/dev/null'
    result = subprocess.run(find_cmd, shell=True, capture_output=True, text=True)
    vendor_json_path = result.stdout.strip().split('\n')[0] # Lấy kết quả đầu tiên

    if vendor_json_path and 'nvidia' in vendor_json_path:
        print(f"   -> Đã tìm thấy tại: {vendor_json_path}")
        os.environ['__EGL_VENDOR_LIBRARY_FILENAMES'] = vendor_json_path

        # --- Bước 3: Khởi tạo ModernGL ---
        print("\n⏳ Bước 3: Đang tạo context ModernGL...")
        try:
            ctx = moderngl.create_standalone_context(backend='egl')
            print("   -> Tạo context thành công!")

            # --- Bước 4: Kiểm tra kết quả ---
            print("\n--- KẾT QUẢ ---")
            renderer = ctx.info.get('GL_RENDERER', '')
            print(f"GL_RENDERER: {renderer}")

            if 'nvidia' in renderer.lower():
                print("\n✅ Tuyệt vời! ModernGL đã kết nối với GPU NVIDIA.")
            else:
                print(f"\n❌ Thất bại! Vẫn đang dùng '{renderer}'.")

        except Exception as e:
            print(f"\n❌ LỖI khi tạo context: {e}")

    else:
        print("\n❌ LỖI: Không thể tìm thấy file 'nvidia_egl.json'. Không thể cấu hình để dùng GPU.")

In [ ]:
# ================================================================
# BƯỚC 1: CÀI ĐẶT CÁC THƯ VIỆN HỆ THỐNG
# (Để kết nối OpenGL với GPU trong môi trường không có màn hình)
# ================================================================
!apt-get update
!apt-get install -y libgl1-mesa-glx libegl1-mesa libosmesa6-dev patchelf mesa-utils
print("✅ Đã cài đặt xong thư viện hệ thống cho OpenGL/EGL.")

# ================================================================
# BƯỚC 2: CÀI ĐẶT THƯ VIỆN PYTHON VÀ ỨNG DỤNG
# ================================================================
!pip install -q pyvista ipywidgets
%cd /content/
!pip install git+https://github.com/BrokenSource/DepthFlow.git -q
print("✅ Đã cài đặt xong PyVista và DepthFlow.")

# ================================================================
# BƯỚC 3: CẤU HÌNH MÔI TRƯỜNG PYTHON VÀ KIỂM TRA
# ================================================================
import os
# Lệnh này báo cho Python sử dụng backend EGL (dành cho GPU server)
os.environ['PYOPENGL_PLATFORM'] = 'egl'

# Chạy lệnh kiểm tra.
# Bây giờ, đầu ra PHẢI là "Tesla T4", không còn là "llvmpipe".
print("\n🔍 Kiểm tra renderer của OpenGL...")
!glxinfo | grep "OpenGL renderer"

# ================================================================
# BƯỚC 4: CHẠY ỨNG DỤNG CỦA BẠN
# ================================================================
print("\n🚀 Khởi động DepthFlow trên GPU...")
!depthflow gradio --share

In [4]:
import torch

print(f"Phiên bản PyTorch: {torch.__version__}")

# Kiểm tra xem CUDA (nền tảng GPU của NVIDIA) có sẵn cho PyTorch không
cuda_available = torch.cuda.is_available()
print(f"CUDA có khả dụng không? (Is CUDA available?): {cuda_available}")

if cuda_available:
    # Lấy số lượng GPU
    device_count = torch.cuda.device_count()
    print(f"Số lượng GPU tìm thấy: {device_count}")

    # In tên của GPU đang được sử dụng
    current_device_name = torch.cuda.get_device_name(torch.cuda.current_device())
    print(f"Tên thiết bị GPU: {current_device_name}")
else:
    print("\n>>> LỖI: PyTorch không tìm thấy GPU. Đây là nguyên nhân chính!")
    print(">>> Lý do phổ biến: Phiên bản PyTorch bạn cài là bản CPU-only hoặc driver/CUDA toolkit chưa tương thích.")

Phiên bản PyTorch: 2.6.0+cu124
CUDA có khả dụng không? (Is CUDA available?): True
Số lượng GPU tìm thấy: 1
Tên thiết bị GPU: Tesla T4
